In [1]:
from tensorflow import keras
from drowsiness_detection import config
from drowsiness_detection.data import session_type_mapping, get_feature_data, \
    preprocess_feature_data, filename_to_session_type_and_id, get_kss_labels_for_feature_file, get_data_for_nn, preprocess_data_for_nn, merge_nn_data
from drowsiness_detection.helpers import binarize
import numpy as np
from pathlib import Path
from tqdm import tqdm


In [2]:
window_size = 10
recording_frequency = 30
config.set_paths(recording_frequency, window_size)
config.PATHS.WINDOW_DATA

PosixPath('/media/tim/My Passport/drowsiness_data/Windows/Windows_30_Hz/WindowData/10_sec')

In [3]:
exclude_by = "a"  # exclude alcohol session

In [8]:
def load_nn_data(exclude_by, data_path: Path = config.PATHS.WINDOW_DATA):
    data_gen  = get_data_for_nn(data_path=data_path)
    data_gen = preprocess_data_for_nn(data_generator=data_gen, exclude_sess_type=exclude_by)
    for _ in range(50):
        next(data_gen)
    return merge_nn_data(data_generator=data_gen)

%time X, y = load_nn_data(data_path=config.PATHS.WINDOW_DATA, exclude_by=exclude_by)

KeyboardInterrupt: 

In [9]:
print(X.shape, y.shape)

(17380, 300, 23) (17380,)
